### Fetching Data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Custom_NER_Model/mtsamples.csv')
df

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
...,...,...,...,...,...,...
4994,4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",NaN
4995,4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun..."
4996,4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema...",NaN
4997,4997,Mother states he has been wheezing and coughing.,Allergy / Immunology,Asthma in a 5-year-old,"CHIEF COMPLAINT: , This 5-year-old male presen...",NaN


In [ ]:
no_of_transcription = len(df['transcription'])
no_of_transcription = 20
for i in range(no_of_transcription):
  txt = df['transcription'][i]
  if type(txt) != str:
    continue
  with open('/content/drive/MyDrive/Custom_NER_Model/transcription.txt', 'a') as f:
    f.write(txt)
    f.write('\n')

### Anotate Data

Use website: https://tecoholic.github.io/ner-annotator/

After anotating, use that to train the ner model

### Building NER model

#### Installing libraries

In [ ]:
!pip install -U spacy
!pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

#### Importing libraries

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

print(spacy.__version__)
!nvidia-smi

3.7.5
Fri Aug  2 17:05:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+-------------------

#### Importing the essential data

In [ ]:
data = json.load(open('/content/drive/MyDrive/Custom_NER_Model/annotations.json', 'r'))
len(data)
data[0]

['SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Neck:  Supple without adenopathy.

#### Initializing Config file

Essentially this file has everything: Architecture, hyperparameters, optimizer info, i.e., almost everything related to a model

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/Custom_NER_Model/base_config.cfg /content/drive/MyDrive/Custom_NER_Model/config/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Custom_NER_Model/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


#### Defining data preprocessing function for NER Model

Basically, to transforms annotated data into a format compatible with spaCy v3, creating Doc objects with character spans linked to entities.


Utilizes spaCy’s DocBin for efficient storage and loading of processed documents, crucial for managing large datasets during model training.

In [ ]:
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass
  return db

#### Data Splitting and Docbin Creation

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

len(train), len(test)

# Open a file to log errors during annotation processing
file = open('/content/drive/MyDrive/Custom_NER_Model/trained_models/train_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/Custom_NER_Model/trained_models/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/Custom_NER_Model/trained_models/test_data.spacy')

# Close the error log file
file.close()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
100%|██████████| 5/5 [00:00<00:00, 54.65it/s]


#### Training NER Model

In [ ]:
!python -m spacy train /content/drive/MyDrive/Custom_NER_Model/config/config.cfg  --output /content/drive/MyDrive/Custom_NER_Model/trained_models/output  --paths.train /content/drive/MyDrive/Custom_NER_Model/trained_models/train_data.spacy  --paths.dev /content/drive/MyDrive/Custom_NER_Model/trained_models/test_data.spacy --gpu-id 0

ℹ Saving to output directory:
/content/drive/MyDrive/Custom_NER_Model/trained_models/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/

#### Testing NER Model

In [ ]:
import spacy_transformers

test_text = "CHIEF COMPLAINT:,  Urinary retention.,HISTORY OF PRESENT ILLNESS: , This is a 66-year-old gentleman status post deceased donor kidney transplant in 12/07, who has had recurrent urinary retention issues since that time.  Most recently, he was hospitalized on 02/04/08 for acute renal insufficiency, which was probably secondary to dehydration.  He was seen by urology again at this visit for urinary retention.  He had been seen by urology during a previous hospitalization and he passed his voiding trial at the time of his stent removal on 01/22/08.  Cystoscopy showed at that time obstructive BPH.  He was started on Flomax at the time of discharge from the hospital.  During the most recent readmission on 02/04/08, he went back into urinary retention and he had had a Foley placed at the outside hospital.,REVIEW OF SYSTEMS:,  Positive for blurred vision, nasal congestion, and occasional constipation.  Denies chest pain, shortness of breath or any rashes or lesions.  All other systems were reviewed and found to be negative.,PAST MEDICAL HISTORY:,1.  End-stage renal disease, now status post deceased donor kidney transplant in 12/07.,2.  Hypertension.,3.  History of nephrolithiasis.,4.  Gout.,5.  BPH.,6.  DJD.,PAST SURGICAL HISTORY:,1.  Deceased donor kidney transplant in 12/07.,2.  Left forearm and left upper arm fistula placements.,FAMILY HISTORY:  ,Significant for mother with an unknown type of cancer, possibly colon cancer or lung and prostate problems on his father side of the family.  He does not know whether his father side of the family had any history of prostate cancer.,HOME MEDICATIONS:,1.  Norvasc.,2.  Toprol 50 mg.,3.  Clonidine 0.2 mg.,4.  Hydralazine.,5.  Flomax.,6.  Allopurinol.,7.  Sodium bicarbonate.,8.  Oxybutynin.,9.  Coumadin.,10.  Aspirin.,11.  Insulin 70/30.,12.  Omeprazole.,13.  Rapamune.,14.  CellCept.,15.  Prednisone.,16.  Ganciclovir.,17.  Nystatin swish and swallow.,18.  Dapsone.,19.  Finasteride.,ALLERGIES:,  No known drug allergies.,PHYSICAL EXAMINATION:,GENERAL:  This is a well-developed, well-nourished male, in no acute distress.  VITAL SIGNS:  Temperature 98, blood pressure 129/72, pulse 96, and weight 175.4 pounds.  LUNGS:  Clear to auscultation bilaterally.  CARDIOVASCULAR:  Regular rate and rhythm with a 3/6 systolic murmur.  ABDOMEN:  Right lower quadrant incision site scar well healed.  Nontender to palpation.  Liver and spleen not enlarged.  No hernias appreciated.  PENIS:  Normal male genitalia.  No lesions appreciated on the penis.  Previous DRE showed the prostate of approximately 40 grams and no nodules.  Foley in place and draining clear urine.,The patient underwent fill and pull study, in which his bladder tolerated 120 ml of sterile water passively filling his bladder.  He spontaneously voided without the Foley 110 mL.,ASSESSMENT AND PLAN:  ,This is a 66-year-old male with signs and symptoms of benign prostatic hypertrophy, who has had recurrent urinary retention since the kidney transplant in 12/07.  He passed his fill and pull study and was thought to self-catheterize in the event that he does incur urinary retention again.  We discussed with Mr. Barker that he has a urologist closer to his home and he lives approximately 3 hours away; however, he desires to continue follow up with the urology clinic at MCG and has been set up for followup in 6 weeks.  He was also given a prescription for 6 months of Flomax and Proscar.  He did not have a PSA drawn today as he had a catheter in place, therefore his PSA could be falsely elevated.  He will have PSA level drawn either just before his visit for followup."

nlp = spacy.load('/content/drive/MyDrive/Custom_NER_Model/trained_models/output/model-best')

doc = nlp(test_text)
# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


66-year-old   ->>>>   AGE
Flomax   ->>>>   CURRENT MEDICATIONS
renal disease,   ->>>>   PAST SURGICAL HISTORY
colon   ->>>>   PATIENT HISTORY
lung and   ->>>>   PATIENT HISTORY
prostate problems   ->>>>   PATIENT HISTORY
prostate cancer.   ->>>>   PAST SURGICAL HISTORY
Norvasc.,2.     ->>>>   ASSESMENT
Toprol 50 mg   ->>>>   TEST RESULTS
Clonidine 0.2 mg.   ->>>>   TEST RESULTS
Sodium bicarbonate.   ->>>>   TEST RESULTS
Omeprazole   ->>>>   MEDICATIONS
Rapamune   ->>>>   CURRENT MEDICATIONS
Prednisone   ->>>>   CURRENT MEDICATIONS
Nystatin   ->>>>   MEDICATIONS
Dapsone   ->>>>   CURRENT MEDICATIONS
Temperature 98, blood pressure 129/72, pulse 96, and weight 175.4 pounds.   ->>>>   TEST RESULTS
Right lower quadrant incision site scar well healed.  Nontender to palpation.  Liver and spleen not enlarged.  No hernias appreciated.  PENIS:  Normal male genitalia.  No lesions appreciated on the penis.  Previous DRE showed the prostate of approximately 40 grams and no nodules.  Foley in place 

NOTE: Need more data and better annotations. Possibly better compute